In [11]:
#!/usr/bin/env python3

import numpy as np
import pandas as pd
import random
import os
import matplotlib.pyplot as plt
from tqdm import trange

DEBUG = False
def dprint(*args):
    if DEBUG:
        print(*args)

# --------------------------------------------------------------
# 1) Valuation with eta (normalized)
# --------------------------------------------------------------
def get_valuation(eta, own_signal, others_signals):
    """
    Valuation with affiliation parameter eta on normalized signals in [0, 1].
    alpha(eta) = 1 - 0.5*eta
    beta(eta)  = 0.5*eta
    
    v_i = alpha(eta)*own_signal + beta(eta)*mean(others_signals)
    Since own_signal and others_signals are in [0,1], v_i remains in [0,1].
    """
    alpha = 1.0 - 0.5 * eta
    beta = 0.5 * eta
    return alpha * own_signal + beta * np.mean(others_signals)

# --------------------------------------------------------------
# 2) Payoffs
# --------------------------------------------------------------
def get_payoffs(bids, valuations, auction_type):
    """
    Calculate payoffs for multiple bidders.
    First-price: winner pays own bid.
    Second-price: winner pays second-highest bid.
    Ties are broken randomly.
    """
    n_bidders = len(bids)
    rewards = np.zeros(n_bidders)
    sorted_indices = np.argsort(bids)[::-1]
    winner = sorted_indices[0]
    highest_bid = bids[winner]

    # Handle ties for the highest bid
    tied_indices = [i for i in sorted_indices if bids[i] == highest_bid]
    if len(tied_indices) > 1:
        winner = random.choice(tied_indices)

    second_highest_bid = bids[sorted_indices[1]] if len(bids) > 1 else highest_bid

    if auction_type == "first":
        rewards[winner] = valuations[winner] - highest_bid
    else:  # second-price
        rewards[winner] = valuations[winner] - second_highest_bid

    return rewards, winner, highest_bid

# --------------------------------------------------------------
# 3) Q-learning update
# --------------------------------------------------------------
def qlearning_update(Q, s, a, r, s_next, alpha, gamma):
    """
    Q-learning update for single state-action pair.
    """
    old_q = Q[s, a]
    best_future = np.max(Q[s_next])
    Q[s, a] = old_q + alpha * (r + gamma * best_future - old_q)

# --------------------------------------------------------------
# 4) Build state index (with normalization)
# --------------------------------------------------------------
def build_state(own_signal, median_opp_bid, past_winning_bid,
                median_opp_past_bid_index, winner_bid_index_state,
                n_val_bins, n_bid_bins):
    """
    Build a state index for Q-table lookups:
    - own_signal in [0, 1], discretized into [0, n_val_bins - 1].
    - median_opp_bid in [0, 1], discretized into [0, n_bid_bins - 1] if included.
    - past_winning_bid in [0, 1], discretized into [0, n_bid_bins - 1] if included.
    
    The final idx is computed based on these discrete values.
    """
    # Discretize own_signal to [0, n_val_bins-1]
    own_signal_idx = int(own_signal * (n_val_bins - 1))

    # Discretize median_opp_bid to [0, n_bid_bins-1] (only if used)
    median_idx = int(median_opp_bid * (n_bid_bins - 1)) if median_opp_past_bid_index else 0

    # Discretize past_winning_bid to [0, n_bid_bins-1] (only if used)
    winner_idx = int(past_winning_bid * (n_bid_bins - 1)) if winner_bid_index_state else 0

    m_size = n_val_bins if median_opp_past_bid_index else 1
    w_size = n_val_bins if winner_bid_index_state else 1

    idx = own_signal_idx
    idx += n_val_bins * median_idx
    idx += (n_val_bins * m_size) * winner_idx

    dprint(f"State index: {idx} (own_signal={own_signal}, median_opp_bid={median_opp_bid}, past_winning_bid={past_winning_bid})")
    return idx

# --------------------------------------------------------------
# 5) Run single experiment
# --------------------------------------------------------------
def run_experiment(eta, auction_type, alpha, gamma, episodes,
                   init, exploration, asynchronous, n_bidders,
                   median_opp_past_bid_index, winner_bid_index_state, seed=0):
    """
    Run Q-learning for one experiment with optional state features,
    using normalized signals, valuations, and states.
    """
    np.random.seed(seed)
    random.seed(seed)

    # Number of bins for signals (value states) and bids
    n_val_bins = 6
    n_bid_bins = 6
    m_size = n_bid_bins if median_opp_past_bid_index else 1
    w_size = n_bid_bins if winner_bid_index_state else 1

    # Total number of states = n_val_bins * m_size * w_size
    n_states = n_val_bins * m_size * w_size

    # Initialize Q-tables
    if init == "random":
        Q = np.random.rand(n_bidders, n_states, n_bid_bins)
    else:  # init == "zeros"
        Q = np.zeros((n_bidders, n_states, n_bid_bins))

    # Actions in [0, 1] range, discretized into n_bid_bins
    actions = np.linspace(0, 1, n_bid_bins)

    # Epsilon schedule
    eps_start, eps_end = 1.0, 0.01
    decay_end = int(0.9 * episodes)

    revenues = []
    past_bids = np.zeros(n_bidders)  # Will store normalized past bids
    past_winner_bid = 0.0            # Will store normalized past winning bid

    def choose_action(Q_row, eps):
        if exploration == "boltzmann":
            ex = np.exp(Q_row)
            probs = ex / np.sum(ex)
            return np.random.choice(len(Q_row), p=probs)
        else:  # e-greedy
            if np.random.rand() < eps:
                return np.random.randint(n_bid_bins)
            return np.argmax(Q_row)

    for ep in range(episodes):
        # Update epsilon
        eps = eps_start - (ep / decay_end) * (eps_start - eps_end) if ep < decay_end else eps_end

        # ----------------------------------------------------
        # 1) Generate normalized signals
        # ----------------------------------------------------
        # Signals in [0, 1] by dividing by (n_val_bins - 1).
        raw_signals = np.random.randint(n_val_bins, size=n_bidders)
        signals = raw_signals / (n_val_bins - 1)  # Now in [0,1]

        # ----------------------------------------------------
        # 2) Calculate valuations (already in [0,1])
        # ----------------------------------------------------
        valuations = [get_valuation(eta, signals[i], np.delete(signals, i)) 
                      for i in range(n_bidders)]

        # ----------------------------------------------------
        # 3) Build states
        # ----------------------------------------------------
        states = [
            build_state(
                own_signal=signals[i],
                median_opp_bid=np.median(np.delete(past_bids, i)),  # median of last round's bids in [0,1]
                past_winning_bid=past_winner_bid,                  # winning bid of last round in [0,1]
                median_opp_past_bid_index=median_opp_past_bid_index,
                winner_bid_index_state=winner_bid_index_state,
                n_val_bins=n_val_bins,
                n_bid_bins=n_bid_bins
            )
            for i in range(n_bidders)
        ]

        # ----------------------------------------------------
        # 4) Choose actions and get bids
        # ----------------------------------------------------
        actions_taken = [choose_action(Q[i, states[i]], eps) for i in range(n_bidders)]
        bids = [actions[a] for a in actions_taken]  # Bids in [0,1]

        # ----------------------------------------------------
        # 5) Calculate payoffs
        # ----------------------------------------------------
        rewards, winner, highest_bid = get_payoffs(bids, valuations, auction_type)

        # ----------------------------------------------------
        # 6) Update Q-tables
        # ----------------------------------------------------
        if asynchronous:
            # Asynchronous update: each bidder's next state is built from a new signal draw
            for i in range(n_bidders):
                # Generate new signals for the "next" state
                raw_next_signals = np.random.randint(n_val_bins, size=n_bidders)
                next_signals = raw_next_signals / (n_val_bins - 1)

                next_state = build_state(
                    own_signal=next_signals[i],
                    median_opp_bid=np.median(np.delete(bids, i)),  # median of current round's bids
                    past_winning_bid=highest_bid,                  # current round's winning bid
                    median_opp_past_bid_index=median_opp_past_bid_index,
                    winner_bid_index_state=winner_bid_index_state,
                    n_val_bins=n_val_bins,
                    n_bid_bins=n_bid_bins
                )
                qlearning_update(Q[i], states[i], actions_taken[i], rewards[i], next_state, alpha, gamma)
        else:
            # Synchronous (counterfactual) updates
            for i in range(n_bidders):
                state = states[i]
                for alt_action in range(n_bid_bins):
                    # Counterfactual: what if bidder i took alt_action instead
                    counterfactual_bids = bids.copy()
                    counterfactual_bids[i] = actions[alt_action]
                    counterfactual_rewards, _, _ = get_payoffs(counterfactual_bids, valuations, auction_type)
                    max_next_q = np.max(Q[i, state])  # Next state is the same in synchronous setting
                    Q[i, state, alt_action] = (1 - alpha) * Q[i, state, alt_action] + \
                                              alpha * (counterfactual_rewards[i] + gamma * max_next_q)

        # ----------------------------------------------------
        # 7) Track results and update memory
        # ----------------------------------------------------
        revenues.append(max(bids))
        past_bids = np.array(bids)      # Store normalized bids
        past_winner_bid = highest_bid   # Store normalized winning bid

    # --------------------------------------------------------
    # 8) Compute final statistics
    # --------------------------------------------------------
    avg_rev_last_1000 = np.mean(revenues[-1000:]) if len(revenues) >= 1000 else np.mean(revenues)
    regrets = [1.0 - r for r in revenues]  # Example: if max possible is 1, then regret = 1 - revenue
    avg_regret_of_seller = np.mean(regrets)

    # Approximate time to converge: first time rolling avg is within ±5% of the final average
    time_to_converge = episodes
    rolling_avg = pd.Series(revenues).rolling(window=1000, min_periods=1).mean()
    lower_bound, upper_bound = 0.95 * avg_rev_last_1000, 1.05 * avg_rev_last_1000
    for t in range(len(rolling_avg)):
        if lower_bound <= rolling_avg.iloc[t] <= upper_bound:
            time_to_converge = t
            break

    return avg_rev_last_1000, time_to_converge, avg_regret_of_seller

# --------------------------------------------------------------
# 6) Main experiment loop
# --------------------------------------------------------------
def main_experiment(K=50):
    """
    Generate data for K experiments with random parameter draws.
    Now uses normalized signals, valuations, and states.
    """
    results = []
    auction_type_options = ["first", "second"]
    init_options = ["random", "zeros"]
    exploration_options = ["egreedy", "boltzmann"]

    for seed in trange(K, desc="Generating experiments"):
        eta = random.uniform(0.0, 1.0)
        alpha = random.uniform(0.01, 0.1)
        gamma = random.uniform(0.0, 0.99)
        episodes = int(random.uniform(10_000, 100_000))
        auction_type = random.choice(auction_type_options)
        init = random.choice(init_options)
        exploration = random.choice(exploration_options)
        asynchronous = random.choice([0, 1])
        n_bidders = random.choice([2, 4, 6])
        median_opp_past_bid_index = random.choice([False, True])
        winner_bid_index_state = random.choice([False, True])

        avg_rev_last_1000, time_to_converge, avg_regret_of_seller = run_experiment(
            eta=eta,
            auction_type=auction_type,
            alpha=alpha,
            gamma=gamma,
            episodes=episodes,
            init=init,
            exploration=exploration,
            asynchronous=asynchronous,
            n_bidders=n_bidders,
            median_opp_past_bid_index=median_opp_past_bid_index,
            winner_bid_index_state=winner_bid_index_state,
            seed=seed
        )

        results.append({
            "eta": eta,
            "alpha": alpha,
            "gamma": gamma,
            "episodes": episodes,
            "auction_type": auction_type,
            "init": init,
            "exploration": exploration,
            "asynchronous": asynchronous,
            "n_bidders": n_bidders,
            "median_opp_past_bid_index": median_opp_past_bid_index,
            "winner_bid_index_state": winner_bid_index_state,
            "avg_rev_last_1000": avg_rev_last_1000,
            "time_to_converge": time_to_converge,
            "avg_regret_of_seller": avg_regret_of_seller
        })
    print(results)
    return pd.DataFrame(results)

# --------------------------------------------------------------
# 7) Run and save data
# --------------------------------------------------------------
if __name__ == "__main__":
    os.makedirs("experiment2", exist_ok=True)

    df = main_experiment(K=1000)
    csv_path = "experiment2/data.csv"
    df.to_csv(csv_path, index=False)
    print(f"Data generation complete. Saved to '{csv_path}'.")

Generating experiments: 100%|██████████| 1000/1000 [2:53:10<00:00, 10.39s/it] 

[{'eta': 0.2372384515770254, 'alpha': 0.08239743981135561, 'gamma': 0.5581499998706037, 'episodes': 79105, 'auction_type': 'second', 'init': 'random', 'exploration': 'egreedy', 'asynchronous': 0, 'n_bidders': 4, 'median_opp_past_bid_index': True, 'winner_bid_index_state': True, 'avg_rev_last_1000': 0.8286, 'time_to_converge': 1, 'avg_regret_of_seller': 0.1606624107199292}, {'eta': 0.995122435179255, 'alpha': 0.08847717166384675, 'gamma': 0.965508831960166, 'episodes': 69883, 'auction_type': 'second', 'init': 'zeros', 'exploration': 'egreedy', 'asynchronous': 1, 'n_bidders': 2, 'median_opp_past_bid_index': False, 'winner_bid_index_state': False, 'avg_rev_last_1000': 0.6762, 'time_to_converge': 6, 'avg_regret_of_seller': 0.3156161011977162}, {'eta': 0.5666643615624456, 'alpha': 0.064036226424076, 'gamma': 0.6490557735143162, 'episodes': 89414, 'auction_type': 'second', 'init': 'zeros', 'exploration': 'boltzmann', 'asynchronous': 0, 'n_bidders': 4, 'median_opp_past_bid_index': False, 'win

In [1]:
#!/usr/bin/env python3

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# DoubleML
import doubleml as dml
from doubleml import DoubleMLData, DoubleMLIRM

# LightGBM
from lightgbm import LGBMRegressor, LGBMClassifier

# For spline expansions (CATE)
import patsy


def main():
    # ------------------------------------------------------------------------------
    # 1) Setup: Create output directory
    # ------------------------------------------------------------------------------
    os.makedirs("experiment2", exist_ok=True)

    # ------------------------------------------------------------------------------
    # 2) Load Experiment 2 Data
    # ------------------------------------------------------------------------------
    df = pd.read_csv("experiment2/data.csv")

    # ------------------------------------------------------------------------------
    # 3) Treatment and Covariates
    # ------------------------------------------------------------------------------
    # Define treatment
    df["D"] = (df["auction_type"] == "first").astype(int)

    # Continuous covariates
    cont_cols = ["eta", "alpha", "gamma", "episodes"]

    # Binary covariates
    df["init"] = (df["init"] == "random").astype(int)
    df["exploration"] = (df["exploration"] == "egreedy").astype(int)
    df["median_opp_past_bid_index"] = df["median_opp_past_bid_index"].astype(int)
    df["winner_bid_index_state"] = df["winner_bid_index_state"].astype(int)
    df["asynchronous"] = df["asynchronous"].astype(int)

    binary_cols = ["init", "exploration", "median_opp_past_bid_index", "winner_bid_index_state", "asynchronous"]

    # All covariates
    X_cols = cont_cols + binary_cols

    # Define outcomes
    outcomes = ["avg_rev_last_1000", "time_to_converge", "avg_regret_of_seller"]

    # ------------------------------------------------------------------------------
    # 4) Loop Over Outcomes
    # ------------------------------------------------------------------------------
    for outcome in outcomes:
        print(f"\n========== Inference for Outcome: {outcome} ==========")

        # Define current outcome
        df["Y"] = df[outcome]

        # ------------------------------------------------------------------------------
        # 5) Prepare DoubleML Data and Model
        # ------------------------------------------------------------------------------
        dml_data = DoubleMLData(
            df,
            y_col="Y",
            d_cols="D",
            x_cols=X_cols
        )

        # Define learners
        ml_g = LGBMRegressor(verbose=-1, random_state=123)
        ml_m = LGBMClassifier(verbose=-1, random_state=123)

        # Initialize DoubleML IRM Model
        dml_irm = DoubleMLIRM(
            dml_data,
            ml_g=ml_g,
            ml_m=ml_m,
            n_folds=3,
            score="ATE"
        )

        # Fit the model
        dml_irm.fit()

        # ------------------------------------------------------------------------------
        # 6) Print and Save ATE Results
        # ------------------------------------------------------------------------------
        print(f"========== ATE Results for {outcome} ==========")
        print(dml_irm.summary)

        # ------------------------------------------------------------------------------
        # 7) GATEs for Binary Covariates
        # ------------------------------------------------------------------------------
        print(f"Generating GATE plots for {outcome}...")
        n_bin = len(binary_cols)
        nrows_gate = int(np.ceil(n_bin / 3))  # up to 3 columns
        ncols_gate = min(n_bin, 3)

        fig_gate, axes_gate = plt.subplots(nrows=nrows_gate, ncols=ncols_gate,
                                           figsize=(5 * ncols_gate, 4 * nrows_gate))

        if n_bin == 1:
            axes_gate = np.array([axes_gate])

        for i, bin_col in enumerate(binary_cols):
            groups_df = df[[bin_col]].astype("category")
            gate_obj = dml_irm.gate(groups=groups_df)
            ci_95_gate = gate_obj.confint(level=0.95)

            effects = ci_95_gate["effect"]
            lower_95 = ci_95_gate["2.5 %"]
            upper_95 = ci_95_gate["97.5 %"]

            ax = axes_gate.flatten()[i] if n_bin > 1 else axes_gate[0]
            x_positions = [0, 1]
            ax.errorbar(
                x_positions, effects,
                yerr=[effects - lower_95, upper_95 - effects],
                fmt="o", capsize=5
            )
            ax.set_title(f"GATE: {bin_col}")
            ax.set_xticks(x_positions)
            ax.set_xticklabels([f"{bin_col}=0", f"{bin_col}=1"])
            ax.set_ylabel("Estimated GATE")

        fig_gate.tight_layout()
        gate_plot_path = os.path.join("experiment2", f"gate_plots_{outcome}.png")
        fig_gate.savefig(gate_plot_path, bbox_inches="tight")
        plt.close(fig_gate)
        print(f"GATE plots for {outcome} saved to: {gate_plot_path}")

        # ------------------------------------------------------------------------------
        # 8) CATEs for Continuous Covariates
        # ------------------------------------------------------------------------------
        print(f"Generating CATE plots for {outcome}...")
        n_cont = len(cont_cols)
        nrows_cate = int(np.ceil(n_cont / 3))
        ncols_cate = min(n_cont, 3)

        fig_cate, axes_cate = plt.subplots(nrows=nrows_cate, ncols=ncols_cate,
                                           figsize=(5 * ncols_cate, 4 * nrows_cate))

        if n_cont == 1:
            axes_cate = np.array([axes_cate])

        for i, cont_col in enumerate(cont_cols):
            design_matrix = patsy.dmatrix(f"bs({cont_col}, df=5, degree=2)", df)
            spline_basis = pd.DataFrame(design_matrix)

            cate_obj = dml_irm.cate(basis=spline_basis)
            ci_95_cate = cate_obj.confint(basis=spline_basis, level=0.95)

            effects_cate = ci_95_cate["effect"].values
            lower_95_cate = ci_95_cate["2.5 %"].values
            upper_95_cate = ci_95_cate["97.5 %"].values

            x_values = df[cont_col].values
            idx_sort = np.argsort(x_values)

            x_sorted = x_values[idx_sort]
            eff_sorted = effects_cate[idx_sort]
            low_sorted = lower_95_cate[idx_sort]
            up_sorted = upper_95_cate[idx_sort]

            ax_cate = axes_cate.flatten()[i] if n_cont > 1 else axes_cate[0]
            ax_cate.plot(x_sorted, eff_sorted, label="CATE")
            ax_cate.fill_between(x_sorted, low_sorted, up_sorted,
                                 alpha=0.2, label="95% CI")
            ax_cate.set_title(f"CATE: {cont_col}")
            ax_cate.set_xlabel(cont_col)
            ax_cate.set_ylabel("Estimated Treatment Effect")
            ax_cate.legend()

        fig_cate.tight_layout()
        cate_plot_path = os.path.join("experiment2", f"cate_plots_{outcome}.png")
        fig_cate.savefig(cate_plot_path, bbox_inches="tight")
        plt.close(fig_cate)
        print(f"CATE plots for {outcome} saved to: {cate_plot_path}")

    print("\nInference complete. All results saved in 'experiment2/' folder.")


if __name__ == "__main__":
    main()



========== Inference for Outcome: avg_rev_last_1000 ==========
========== ATE Results for avg_rev_last_1000 ==========
       coef   std err         t         P>|t|     2.5 %    97.5 %
D -0.131025  0.019159 -6.838727  7.989993e-12 -0.168577 -0.093474
Generating GATE plots for avg_rev_last_1000...
GATE plots for avg_rev_last_1000 saved to: experiment2/gate_plots_avg_rev_last_1000.png
Generating CATE plots for avg_rev_last_1000...
CATE plots for avg_rev_last_1000 saved to: experiment2/cate_plots_avg_rev_last_1000.png

========== Inference for Outcome: time_to_converge ==========
========== ATE Results for time_to_converge ==========
          coef      std err         t         P>|t|         2.5 %  \
D  17771.62868  1826.652918  9.729067  2.266622e-22  14191.454749   

         97.5 %  
D  21351.802612  
Generating GATE plots for time_to_converge...
GATE plots for time_to_converge saved to: experiment2/gate_plots_time_to_converge.png
Generating CATE plots for time_to_converge...
CATE pl

In [1]:
#!/usr/bin/env python3

import numpy as np
import pandas as pd
import doubleml
import matplotlib.pyplot as plt
import os
import doubleml as dml
from doubleml import DoubleMLData, DoubleMLIRM
from lightgbm import LGBMRegressor, LGBMClassifier
import patsy
from scipy.stats import norm, pearsonr
from tabulate import tabulate
import math

if __name__ == "__main__":
    # ------------------------------------------------------------------------------
    # 1) Variable definitions and reading data
    # ------------------------------------------------------------------------------
    var_definitions = {
        "eta": "Affiliation parameter (0.0 to 1.0).",
        "alpha": "Learning rate for Q-updates (0.01 to 0.1).",
        "gamma": "Discount factor for future rewards (0.0 to 0.99).",
        "episodes": "Total number of training episodes (10k to 100k).",
        "auction_type": "Treatment: 0 = first-price, 1 = second-price.",
        "init": "Q-initialization: 'random' or 'zeros'.",
        "exploration": "Exploration strategy: 'egreedy' or 'boltzmann'.",
        "asynchronous": "Update mode: 0 = synchronous, 1 = asynchronous.",
        "n_bidders": "Number of bidders (2, 4, or 6).",
        "median_opp_past_bid_index": "Use median of opponents' bids in state?",
        "winner_bid_index_state": "Track winning bid index in state?",
        "avg_rev_last_1000": "Mean seller revenue in the final 1000 episodes.",
        "time_to_converge": "Fraction of episodes until ±5% convergence.",
        "avg_regret_of_seller": "Average regret for the seller (max valuation = 1)."
    }

    df_var_defs = pd.DataFrame([{"Parameter": k, "Definition": v} for k, v in var_definitions.items()])
    print("\n=== Variable Definitions ===")
    print(tabulate(df_var_defs, headers="keys", tablefmt="github"))

    # Load data
    df = pd.read_csv("experiment2/data.csv")

    # Recode auction_type: 'second' -> 1, 'first' -> 0
    df["auction_type"] = (df["auction_type"] == "second").astype(int)

    # Convert time_to_converge to fraction of total episodes
    df["time_to_converge"] = df["time_to_converge"] / df["episodes"]

    # ------------------------------------------------------------------------------
    # 2) Set up columns
    # ------------------------------------------------------------------------------
    treatment_col = "auction_type"
    covariates_list = [
        "eta", "alpha", "gamma", "episodes", "init",
        "exploration", "asynchronous", "n_bidders",
        "median_opp_past_bid_index", "winner_bid_index_state"
    ]
    outcomes_list = ["avg_rev_last_1000", "time_to_converge", "avg_regret_of_seller"]

    # Ensure binary covariates are int
    for b in ["init", "exploration", "asynchronous", "median_opp_past_bid_index", "winner_bid_index_state"]:
        if df[b].dtype == bool:
            df[b] = df[b].astype(int)
        if df[b].dtype not in [np.float64, np.int64, float, int]:
            df[b], _ = pd.factorize(df[b])

    # ------------------------------------------------------------------------------
    # 3) Summary statistics
    # ------------------------------------------------------------------------------
    cols_for_summary = covariates_list + [treatment_col] + outcomes_list
    summary_stats = df[cols_for_summary].describe().T.drop("count", axis=1, errors="ignore")
    print("\n=== Summary Statistics ===")
    print(tabulate(summary_stats, headers="keys", tablefmt="github"))

    # ------------------------------------------------------------------------------
    # 4) Correlations with treatment
    # ------------------------------------------------------------------------------
    corr_results = []
    for col in cols_for_summary:
        if col != treatment_col:
            r, p = pearsonr(df[col], df[treatment_col])
            corr_results.append({"Variable": col, "Correlation": r, "p-value": p})

    print("\n=== Correlations with auction_type ===")
    print(tabulate(corr_results, headers="keys", tablefmt="github"))

    # ------------------------------------------------------------------------------
    # 5) DoubleML Analysis: ATE, GATE, CATE
    # ------------------------------------------------------------------------------
    os.makedirs("experiment2", exist_ok=True)

    for outcome in outcomes_list:
        # Prepare for DoubleML
        df["Y"] = df[outcome]
        dml_data = doubleml.DoubleMLData(
            df, y_col="Y", d_cols=treatment_col, x_cols=covariates_list
        )
        ml_g = LGBMRegressor(random_state=123, verbose=-1)
        ml_m = LGBMClassifier(random_state=123, verbose=-1)
        dml_irm = doubleml.DoubleMLIRM(
            dml_data, ml_g=ml_g, ml_m=ml_m, n_folds=10, n_rep=1, score="ATE"
        )
        dml_irm.fit()

        print(f"\n========== {outcome.upper()} | ATE Results ==========")
        print(dml_irm.summary)

        # --------------------------------------
        # GATE for binary covariates
        # --------------------------------------
        binary_covs = [c for c in covariates_list if df[c].nunique() == 2]
        n_bin = len(binary_covs)
        nrows_gate = math.ceil(n_bin / 3)
        ncols_gate = min(n_bin, 3)
        fig_gate, axes_gate = plt.subplots(nrows=nrows_gate, ncols=ncols_gate,
                                           figsize=(5 * ncols_gate, 4 * nrows_gate))
        if n_bin == 1:
            axes_gate = np.array([axes_gate])
        gate_results = []

        for i, bin_col in enumerate(binary_covs):
            groups_df = df[[bin_col]].astype("category")
            gate_obj = dml_irm.gate(groups=groups_df)
            ci_95 = gate_obj.confint(level=0.95)
            eff = ci_95["effect"]
            lo = ci_95["2.5 %"]
            hi = ci_95["97.5 %"]
            gate_sum = gate_obj.summary
            errs = gate_sum["std err"]

            ax = axes_gate.flatten()[i] if n_bin > 1 else axes_gate[0]
            x_pos = np.arange(len(eff))
            ax.errorbar(x_pos, eff, yerr=[eff - lo, hi - eff], fmt="o", capsize=5)
            ax.set_title(f"GATE: {bin_col} ({outcome})")
            ax.set_xticks(x_pos)
            ax.set_xticklabels([f"{bin_col}={lvl}" for lvl in range(len(eff))])
            ax.set_ylabel("Estimated GATE")

            # Two-group difference t-test
            if len(eff) == 2:
                dval = eff.iloc[1] - eff.iloc[0]
                dvar = errs.iloc[1]**2 + errs.iloc[0]**2
                dse = math.sqrt(dvar)
                tval = dval / dse
                pval = 2.0 * (1.0 - norm.cdf(abs(tval)))
                gate_results.append({
                    "Variable": bin_col,
                    "Group0_Effect": f"{eff.iloc[0]:.4f}",
                    "Group1_Effect": f"{eff.iloc[1]:.4f}",
                    "Diff(Group1-Group0)": f"{dval:.4f}",
                    "StdErr(Diff)": f"{dse:.4f}",
                    "t-value": f"{tval:.4f}",
                    "p-value": f"{pval:.4f}"
                })

        fig_gate.tight_layout()
        gate_path = os.path.join("experiment2", f"gate_plots_{outcome}.png")
        fig_gate.savefig(gate_path, bbox_inches="tight")
        plt.close(fig_gate)

        if gate_results:
            print(f"\nT-tests for GATE ({outcome}):")
            print(tabulate(gate_results, headers="keys", tablefmt="github"))

        # --------------------------------------
        # CATE for continuous covariates
        # --------------------------------------
        cont_covs = [c for c in covariates_list if df[c].nunique() > 2]
        n_cont = len(cont_covs)
        nrows_cate = math.ceil(n_cont / 3)
        ncols_cate = min(n_cont, 3)
        fig_cate, axes_cate = plt.subplots(nrows=nrows_cate, ncols=ncols_cate,
                                           figsize=(5 * ncols_cate, 4 * nrows_cate))
        if n_cont == 1:
            axes_cate = np.array([axes_cate])

        for i, cont_col in enumerate(cont_covs):
            design_matrix = patsy.dmatrix(f"bs({cont_col}, df=5, degree=2)", df)
            spline_basis = pd.DataFrame(design_matrix)
            cate_obj = dml_irm.cate(basis=spline_basis)
            ci_95_cate = cate_obj.confint(basis=spline_basis, level=0.95)
            eff_cate = ci_95_cate["effect"].values
            lo_cate = ci_95_cate["2.5 %"].values
            hi_cate = ci_95_cate["97.5 %"].values

            xvals = df[cont_col].values
            idx_sort = np.argsort(xvals)
            x_sort = xvals[idx_sort]
            eff_sort = eff_cate[idx_sort]
            lo_sort = lo_cate[idx_sort]
            hi_sort = hi_cate[idx_sort]

            axc = axes_cate.flatten()[i] if n_cont > 1 else axes_cate[0]
            axc.plot(x_sort, eff_sort, label="CATE")
            axc.fill_between(x_sort, lo_sort, hi_sort, alpha=0.2, label="95% CI")
            axc.set_title(f"CATE: {cont_col} ({outcome})")
            axc.set_xlabel(cont_col)
            axc.set_ylabel("Estimated TE")
            axc.legend()

        fig_cate.tight_layout()
        cate_path = os.path.join("experiment2", f"cate_plots_{outcome}.png")
        fig_cate.savefig(cate_path, bbox_inches="tight")
        plt.close(fig_cate)

    print("\nAll analysis complete. Results saved in 'experiment2/' directory.")



=== Variable Definitions ===
|    | Parameter                 | Definition                                         |
|----|---------------------------|----------------------------------------------------|
|  0 | eta                       | Affiliation parameter (0.0 to 1.0).                |
|  1 | alpha                     | Learning rate for Q-updates (0.01 to 0.1).         |
|  2 | gamma                     | Discount factor for future rewards (0.0 to 0.99).  |
|  3 | episodes                  | Total number of training episodes (10k to 100k).   |
|  4 | auction_type              | Treatment: 0 = first-price, 1 = second-price.      |
|  5 | init                      | Q-initialization: 'random' or 'zeros'.             |
|  6 | exploration               | Exploration strategy: 'egreedy' or 'boltzmann'.    |
|  7 | asynchronous              | Update mode: 0 = synchronous, 1 = asynchronous.    |
|  8 | n_bidders                 | Number of bidders (2, 4, or 6).                    |
| 